In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn import *
import gc

In [2]:
! ls ../data/*.csv

ls: cannot access '../data/*.csv': No such file or directory


# Data loading

In [3]:
from pathlib import Path
PATH = Path('../../data')

In [4]:
train = pd.read_csv(PATH/'train.csv')
test = pd.read_csv(PATH/'test.csv')

In [5]:
def feature_atomtype(df, s):
    # https://www.kaggle.com/jazivxt/all-this-over-a-dog
    df['atom1'] = df['type'].map(lambda x: str(x)[2])
    df['atom2'] = df['type'].map(lambda x: str(x)[3])
    lbl = preprocessing.LabelEncoder()
    for i in range(4):
        df['type'+str(i)] = lbl.fit_transform(df['type'].map(lambda x: str(x)[i]))

    df = pd.merge(df, s.rename(columns={'atom_index':'atom_index_0', 'x':'x0', 'y':'y0', 'z':'z0', 'atom':'atom1'}), how='left', on=['molecule_name', 'atom_index_0', 'atom1'])
    df = pd.merge(df, s.rename(columns={'atom_index':'atom_index_1', 'x':'x1', 'y':'y1', 'z':'z1', 'atom':'atom2'}), how='left', on=['molecule_name', 'atom_index_1', 'atom2'])
    return df

def feature_pair_geometry(df):
    p0 = df[['x0', 'y0', 'z0']].values
    p1 = df[['x1', 'y1', 'z1']].values
    r = np.linalg.norm(p0 - p1, axis=1)
    df['dist'] = r

    for agg in ['min', 'max', 'mean']:
        tmp = eval('df.groupby(["type"], as_index=False).dist.' + agg + '()')
        tmp.rename(columns={"dist":agg + "_dist"}, inplace=True)
        df = pd.merge(df, tmp, how='left', on=['type'])
    return df

In [8]:
def reduce_mem_usage(df, verbose=True):
    # somewhere from kaggle kernel
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem)
 / start_mem))
    
    return df

In [7]:
def feature_basic(df):
    structures = pd.read_csv(PATH/'structures.csv')
    df = feature_atomtype(df, structures)
    df = feature_pair_geometry(df)
    df = reduce_mem_usage(df)
    del structures
    gc.collect()
    return df

In [8]:
train = feature_basic(train)
test = feature_basic(test)

Mem. usage decreased to 417.58 Mb (48.9% reduction)
Mem. usage decreased to 215.05 Mb (48.9% reduction)


In [9]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom1,atom2,type0,type1,...,x0,y0,z0,x1,y1,z1,dist,min_dist,max_dist,mean_dist
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807602,H,C,0,0,...,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,1.060901,1.247942,1.092900
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000,H,H,1,0,...,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,1.513358,1.969340,1.774895
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254800,H,H,1,0,...,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,1.513358,1.969340,1.774895
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300,H,H,1,0,...,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,1.513358,1.969340,1.774895
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807404,H,C,0,0,...,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,1.060901,1.247942,1.092900


In [10]:
# https://www.kaggle.com/adrianoavelar/bond-calculaltion-lb-0-82
def feature_neighbors(s):
    i_atom = s['atom_index'].values
    i_atom_type = s['atom'].values
    p = s[['x', 'y', 'z']].values
    m = s['molecule_name'].values
    t = np.empty((len(s)+1), dtype=np.object)
    t[:len(s)] = s['atom'].values
    p_compare = p
    m_compare = m
    t_compare = t
    source_row = np.arange(len(s))
    max_atoms = max(s.groupby('molecule_name').atom_index.max().values)
    bonds = np.zeros((len(s)+1, max_atoms+1), dtype=np.int8)
    bond_dists = np.zeros((len(s)+1, max_atoms+1), dtype=np.float32)
    bond_atoms = np.empty((len(s)+1, max_atoms+1), dtype=np.object)
    for i in tqdm(range(max_atoms-1)):
        p_compare = np.roll(p_compare, -1, axis=0)
        m_compare = np.roll(m_compare, -1, axis=0)
        t_compare = np.roll(t_compare, -1, axis=0)

        mask = np.where(m == m_compare, 1, 0) #Are we still comparing atoms in the same molecule?
        dists = np.linalg.norm(p - p_compare, axis=1) * mask
        r_bond = 3.0

        bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)

        source_row = source_row
        target_row = source_row + i + 1 #Note: Will be out of bounds of bonds array for some values of i
        target_row = np.where(np.logical_or(target_row > len(s), mask==0), len(s), target_row) #If invalid target, write to dummy row

        source_atom = i_atom
        target_atom = i_atom + i + 1 #Note: Will be out of bounds of bonds array for some values of i
        target_atom = np.where(np.logical_or(target_atom > max_atoms, mask==0), max_atoms, target_atom) #If invalid target, write to dummy col

        target_atom_type = np.where(np.logical_or(target_row > len(s), mask==0), '', t[target_row]) #If invalid target, write to dummy row    
        source_atom_type = i_atom_type

        bonds[(source_row, target_atom)] = bond
        bonds[(target_row, source_atom)] = bond
        bond_dists[(source_row, target_atom)] = dists
        bond_dists[(target_row, source_atom)] = dists
        bond_atoms[(source_row, target_atom)] = target_atom_type
        bond_atoms[(target_row, source_atom)] = source_atom_type

    bonds = np.delete(bonds, axis=0, obj=-1) #Delete dummy row
    bonds = np.delete(bonds, axis=1, obj=-1) #Delete dummy col
    bond_dists = np.delete(bond_dists, axis=0, obj=-1) #Delete dummy row
    bond_dists = np.delete(bond_dists, axis=1, obj=-1) #Delete dummy col
    bond_atoms = np.delete(bond_atoms, axis=0, obj=-1) #Delete dummy row
    bond_atoms = np.delete(bond_atoms, axis=1, obj=-1) #Delete dummy col
    
    mask = bonds == 1
    bond_lengths_mean = [np.mean(row[mask[j]]) for j,row in enumerate(tqdm(bond_dists))]
    n_bonds = np.sum(bonds, axis=1)
    bond_data = {'n_bonds':n_bonds, 'bond_lengths_mean': bond_lengths_mean }
    bond_df = pd.DataFrame(bond_data)
    s = s.join(bond_df)
    
    atom_types = ['C', 'N', 'H', 'O', 'F']
    bond_data = {}
    for at in atom_types:
        bonds_at = np.empty((len(s), max_atoms), dtype=np.int8) 
        bonds_at[:] = -1
        for i in tqdm(range(len(bond_atoms))):
            mask = bond_atoms[i,:] == at
            atom_j_indices = np.argwhere(mask)
            dists = bond_dists[i, mask]
            atom_j_sorted = np.argsort(dists)
            bonds_at[i, :len(atom_j_sorted)] = atom_j_sorted
            
        maxatom = np.max(np.sum(bonds_at >= 0, axis=1))
        maxatom = 6
        for i in range(maxatom):
            bond_data['bond_%s_%d' % (at, i)] = bonds_at[:, i]
    bond_df = pd.DataFrame(bond_data)
    s = s.join(bond_df)
    return s

In [11]:
def feature_dist(df, s):
    atom0_xyz = df[['x0', 'y0', 'z0']].values
    atom1_xyz = df[['x1', 'y1', 'z1']].values
    for col in tqdm(df.columns):
        if col.startswith('bond') and not col.startswith('bond_length'):
            xyz = pd.merge(df, s.drop('atom', axis=1).rename(columns={'atom_index':col, 'x':col+'_x', 'y':col+'_y', 'z':col+'_z'}), how='left', on=['molecule_name', col])[[col+'_x', col+'_y', col+'_z']].values
            r0 = np.linalg.norm(atom0_xyz - xyz, axis=1)
            r1 = np.linalg.norm(atom1_xyz - xyz, axis=1)
            df[col+'_r0'] = r0
            df[col+'_r1'] = r1
    return df

In [12]:
def feature_bonds(df1, df2):
    structures = pd.read_csv(PATH/'structures.csv')
    structures = feature_neighbors(structures)
    df1 = pd.merge(df1, structures.drop(['x', 'y', 'z'], axis=1).rename(columns={'atom_index': 'atom_index_0', 'atom':'atom1'}), how='left', on=['molecule_name', 'atom_index_0', 'atom1'])
    df1 = pd.merge(df1, structures.drop(['x', 'y', 'z'], axis=1).rename(columns={'atom_index': 'atom_index_0', 'atom':'atom1'}), how='left', on=['molecule_name', 'atom_index_0', 'atom1'])
    df2 = pd.merge(df2, structures.drop(['x', 'y', 'z'], axis=1).rename(columns={'atom_index': 'atom_index_0', 'atom':'atom1'}), how='left', on=['molecule_name', 'atom_index_0', 'atom1'])
    df2 = pd.merge(df2, structures.drop(['x', 'y', 'z'], axis=1).rename(columns={'atom_index': 'atom_index_0', 'atom':'atom1'}), how='left', on=['molecule_name', 'atom_index_0', 'atom1'])
    
    df1 = feature_dist(df1, structures)
    df2 = feature_dist(df2, structures)

    df1 = reduce_mem_usage(df1)
    df2 = reduce_mem_usage(df2)
    del structures
    gc.collect()
    return df1, df2

In [13]:
train, test = feature_bonds(train, test)

100%|██████████| 85/85 [03:29<00:00,  5.33s/it]


Mem. usage decreased to 2931.95 Mb (42.4% reduction)
Mem. usage decreased to 1682.19 Mb (38.4% reduction)


In [14]:
# rdkit & xyz2mol
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
from rdkit.Chem import Draw
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions #Only needed if modifying defaults
DrawingOptions.bondLineWidth=1.8
from rdkit.Chem.rdmolops import SanitizeFlags

# https://github.com/jensengroup/xyz2mol
from xyz2mol import xyz2mol, xyz2AC, AC2mol, read_xyz_file
from pathlib import Path
import pickle

def chiral_stereo_check(mol):
    # avoid sanitization error e.g., dsgdb9nsd_037900.xyz
    Chem.SanitizeMol(mol, SanitizeFlags.SANITIZE_ALL - SanitizeFlags.SANITIZE_PROPERTIES)
    Chem.DetectBondStereochemistry(mol,-1)
    # ignore stereochemistry for now
    #Chem.AssignStereochemistry(mol, flagPossibleStereoCenters=True, force=True)
    #Chem.AssignAtomChiralTagsFromStructure(mol,-1)
    return mol

def xyz2mol(atomicNumList,charge,xyz_coordinates,charged_fragments,quick):
    AC,mol = xyz2AC(atomicNumList,xyz_coordinates)
    new_mol = AC2mol(mol,AC,atomicNumList,charge,charged_fragments,quick)
    new_mol = chiral_stereo_check(new_mol)
    return new_mol

def MolFromXYZ(filename):
    charged_fragments = True
    quick = True
    cache_filename = filename.parent/f'{filename.stem}.pkl'
    if cache_filename.exists():
        return pickle.load(open(cache_filename, 'rb'))
    else:
        try:
            atomicNumList, charge, xyz_coordinates = read_xyz_file(filename)
            mol = xyz2mol(atomicNumList, charge, xyz_coordinates, charged_fragments, quick)
            pickle.dump(mol, open(cache_filename, 'wb'))
        except:
            print(filename)
    return mol

#mol = MolFromXYZ(xyzfiles[1])
#m = Chem.MolFromSmiles(Chem.MolToSmiles(mol, allHsExplicit=True)); m

from multiprocessing import Pool
from tqdm import *
from glob import glob

def MolFromXYZ_(filename):
    return filename.stem, MolFromXYZ(filename)

# skipping below for saving memory
"""
mols = {}
n_cpu = 4
with Pool(n_cpu) as p:
    molecule_names = np.concatenate([train.molecule_name.unique(), test.molecule_name.unique()])
    xyzfiles = [Path(PATH/f'structures/{f}.xyz') for f in molecule_names]
    n = len(xyzfiles)
    with tqdm(total=n) as pbar:
        for res in p.imap_unordered(MolFromXYZ_, xyzfiles):
            mols[res[0]] = res[1]
            pbar.update()
"""

"\nmols = {}\nn_cpu = 4\nwith Pool(n_cpu) as p:\n    molecule_names = np.concatenate([train.molecule_name.unique(), test.molecule_name.unique()])\n    xyzfiles = [Path(PATH/f'structures/{f}.xyz') for f in molecule_names]\n    n = len(xyzfiles)\n    with tqdm(total=n) as pbar:\n        for res in p.imap_unordered(MolFromXYZ_, xyzfiles):\n            mols[res[0]] = res[1]\n            pbar.update()\n"

In [15]:
import dask.dataframe as dd
import multiprocessing
import time

def feature_atom(row):
    m = MolFromXYZ(PATH/f'structures/{row.molecule_name}.xyz')
    #m = mols[row.molecule_name]
    atom = m.GetAtomWithIdx(row.atom_index)
    prop = {}
    nb = [a.GetSymbol() for a in atom.GetNeighbors()] # neighbor atom type symbols
    nb_h = sum([_ == 'H' for _ in nb])
    nb_o = sum([_ == 'O' for _ in nb])
    nb_c = sum([_ == 'C' for _ in nb])
    nb_n = sum([_ == 'N' for _ in nb])
    nb_f = sum([_ == 'F' for _ in nb])
    row['degree'] = atom.GetDegree()
    row['hybridization'] = int(atom.GetHybridization())
    row['inring'] = int(atom.IsInRing())
    row['inring4'] = int(atom.IsInRingSize(4))
    row['inring5'] = int(atom.IsInRingSize(5))
    row['inring6'] = int(atom.IsInRingSize(6))
    row['nb_h'] = nb_h
    row['nb_o'] = nb_o
    row['nb_c'] = nb_c
    row['nb_n'] = nb_n
    row['nb_f'] = nb_f
    return row

def feature_structure_atoms(s):
    names = s.molecule_name.unique()
    meta = s.iloc[:1].apply(feature_atom, axis=1)
    t0 = time.time()
    structures = dd.from_pandas(s, npartitions=4*multiprocessing.cpu_count()) \
                    .map_partitions(lambda df: df.apply(feature_atom, axis=1), meta=meta) \
                    .compute(scheduler='processes')
    print(f'took {time.time() - t0} sec')
    return structures

In [16]:
def feature_bonded_atoms(df, structures):
    bc = [col for col in df.columns if col.startswith('bond') and not col.startswith('bond_length') and not col.endswith('r0') and not col.endswith('r1')]
    for col in tqdm(bc):
        s = structures.drop(['x', 'y', 'z', 'atom'], axis=1).rename(columns={'atom_index': col})
        sc = [_ for _ in s.columns if _ != col and _ != 'molecule_name']
        s = s.rename(columns={k: col+'_'+k for k in sc})
        df = pd.merge(df, s, how='left', on=['molecule_name', col])
        for c in sc:
            df[col+'_'+c] = df[col+'_'+c].fillna(-1).astype(np.int16)
    return df

def feature_atoms(df1, df2):
    if (PATH/'structures_atoms.csv').exists():
        structures = pd.read_csv(PATH/'structures_atoms.csv')
        if 'Unnamed: 0' in structures:
            structures = structures.drop('Unnamed: 0', axis=1)
    else:
        structures = pd.read_csv(PATH/'structures.csv')
        structures = feature_atoms(structures)
        structures.to_csv(PATH/'structures_atoms.csv', index=False)
    
    df1 = feature_bonded_atoms(df1, structures)
    df2 = feature_bonded_atoms(df2, structures)
    df1 = reduce_mem_usage(df1)
    df2 = reduce_mem_usage(df2)
    del structures
    gc.collect()
    return df1, df2

train, test = feature_atoms(train, test)

100%|██████████| 60/60 [04:40<00:00,  7.30s/it]


Mem. usage decreased to 5863.91 Mb (33.3% reduction)
Mem. usage decreased to 3259.24 Mb (32.6% reduction)


In [17]:
train.to_pickle(PATH/'train_tmp.pkl')
test.to_pickle(PATH/'test_tmp.pkl')

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn import *
import gc
from pathlib import Path
PATH = Path('../../data')

In [2]:
train = pd.read_pickle(PATH/'train_tmp.pkl')
test = pd.read_pickle(PATH/'test_tmp.pkl')
# n = 4658147
# train_index = np.array([i for i in range(n) if i % 10 != 0])
# test_index = np.array([i for i in range(n) if i % 10 == 0])
# X_train = pd.read_pickle(PATH/'train_tmp.pkl').iloc[train_index].fillna(-1)
# X_test = pd.read_pickle(PATH/'train_tmp.pkl').iloc[test_index].fillna(-1)

In [26]:
excluded = ['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom1', 'atom2'] + ['x0', 'y0', 'z0', 'x1', 'y1', 'z1']
for i in range(6):
    excluded += ['bond_%s_%d_x' % (t, i) for t in ['C', 'H', 'O', 'N', 'F']]
    excluded += ['bond_%s_%d_y' % (t, i) for t in ['C', 'H', 'O', 'N', 'F']]
col = [c for c in train.columns if c not in ['scalar_coupling_constant'] + excluded]
#reg = ensemble.ExtraTreesRegressor(n_jobs=-1, n_estimators=5, random_state=4, verbose=1)

In [29]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train[col], train['scalar_coupling_constant'], test_size=0.2)
# y_train = X_train['scalar_coupling_constant']
# y_test = X_test['scalar_coupling_constant']
# X_train = X_train[col]
# X_test = X_test[col]

In [30]:
X_train = X_train.fillna(-1)
X_test = X_test.fillna(-1)

In [31]:
reg = ensemble.ExtraTreesRegressor(n_jobs=-1, n_estimators=5, random_state=4, verbose=1)
reg.fit(X_train.drop(['type'], axis=1), y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 105.1min finished


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
          oob_score=False, random_state=4, verbose=1, warm_start=False)

In [32]:
y_pred = reg.predict(X_test.drop('type', axis=1))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:    5.1s finished


In [33]:
# https://www.kaggle.com/uberkinder/efficient-metric
def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [34]:
group_mean_log_mae(y_test, y_pred, X_test.type)

-0.49973494851025835

In [35]:
for t in train.type.unique():
    idx = X_test.type == t
    print(t, (y_test[idx] - y_pred[idx]).abs().mean())

1JHC 1.5427883354914589
2JHH 0.34006164474682826
1JHN 1.104168635526701
2JHN 0.5175124004509403
2JHC 0.7007759737563171
3JHH 0.4122990452001901
3JHC 0.6355013892338905
3JHN 0.33343774004197446


In [37]:
# train with all data
reg.fit(train[col].drop('type', axis=1), train['scalar_coupling_constant'])

/home/sunhwan/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.8s finished


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
          oob_score=False, random_state=4, verbose=1, warm_start=False)

In [41]:
test['scalar_coupling_constant']  = reg.predict(test[col].drop('type', axis=1))
test[['id', 'scalar_coupling_constant']].to_csv('submission.csv', index=False) #float_format='%.9f'

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:    0.1s finished


In [46]:
from IPython.display import FileLink
FileLink('submission.csv')

/home/sunhwan/work/kaggle/champs-scalar-coupling/nbs/submission.csv

In [ ]:
# score: ~0.7